In [1]:
import sys
sys.path.append("../")
sys.path.append("../../")

In [2]:
import numpy as np
from Onsager_calc_db import *
import onsager.crystal as crystal
from states import *
from stars import *
from vector_stars import *
from representations import *
import time
import pickle
from scipy.constants import physical_constants
kB = physical_constants['Boltzmann constant in eV/K'][0]
from matplotlib import pyplot as plt
import scipy.stats as spstats

/usr/local/lib/python3.6/dist-packages/onsager-1.3.3-py3.6.egg/onsager/cluster.py:835: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.


In [3]:
# make a BCC lattice
# We'll modify the jumpnetwork to keep only the 60 degree reorientational jumps.
latt = np.array([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]]) * 0.2831
Fe = crystal.Crystal(latt, [[np.array([0., 0., 0.]), np.array([0.5, 0.5, 0.5])]], ["Fe"])
# Now give it the orientations - for BCC it's [110]
o = np.array([1.,1.,0.])/np.linalg.norm(np.array([1.,1.,0.]))*0.126*2
famp0 = [o.copy()]
family = [famp0]
pdbcontainer_fe = dbStates(Fe, 0, family)
mdbcontainer_fe = mStates(Fe, 0, family)
jset0, jset2 = pdbcontainer_fe.jumpnetwork(0.26, 0.01, 0.01), mdbcontainer_fe.jumpnetwork(0.26, 0.01, 0.01)
print(Fe)

#Lattice:
  a1 = [0.14155 0.14155 0.14155]
  a2 = [-0.14155  0.14155 -0.14155]
  a3 = [-0.14155 -0.14155  0.14155]
#Basis:
  (Fe) 0.0 = [0. 0. 0.]


In [4]:
# Modify jnet0
jnet0 = jset0[0]
jnet0_indexed = jset0[1]
# Let's try to sort the jumps according to closest distance
# except rotational jumps, we don't want them.

def sortkey(entry):
    jmp = jnet0[entry][0]
    or1 = pdbcontainer_fe.iorlist[jmp.state1.iorind][1]
    or2 = pdbcontainer_fe.iorlist[jmp.state2.iorind][1]
    dx = disp(pdbcontainer_fe, jmp.state1, jmp.state2)
    dx1 = np.linalg.norm(jmp.c1*or1/2.)
    dx2 = np.linalg.norm(dx + jmp.c2*or2/2. - jmp.c1*or1/2.)
    dx3 = np.linalg.norm(-jmp.c2*or2/2.)
    return dx1+dx2+dx3

z = np.zeros(3)
indices = []
for jt, jlist in enumerate(jnet0):
    if np.allclose(jnet0_indexed[jt][0][1], z):
        continue
    indices.append(jt)

ind_sort = sorted(indices, key=sortkey)[:1]
len(ind_sort)

1

In [5]:
print(jnet0[ind_sort[0]][0])

Jump object:
Initial state:
	dumbbell : (i, or) index = 5, lattice vector = [0 0 0]
Final state:
	dumbbell : (i, or) index = 4, lattice vector = [1 0 0]
Jumping from c1 = 1 to c2 = -1



In [6]:
pdbcontainer_fe.iorlist

[(0, array([ 1.78190909e-01,  1.78190909e-01, -1.01947883e-17])),
 (0, array([ 0.17819091, -0.17819091,  0.        ])),
 (0, array([ 0.        ,  0.17819091, -0.17819091])),
 (0, array([-1.78190909e-01, -1.01947883e-17,  1.78190909e-01])),
 (0, array([-1.01947883e-17,  1.78190909e-01,  1.78190909e-01])),
 (0, array([0.17819091, 0.        , 0.17819091]))]

In [7]:
# take the jumps we want.
jset0new = ([jnet0[i] for i in ind_sort], [jnet0_indexed[i] for i in ind_sort])

In [8]:
# Now, we modify the mixed dumbbell jumpnetwork.
# We'll include all the jumps there are in the paper
# They are: First and second lowest transition jumps, 60 deg on-site rotation, 90-deg onsite rotation.
jnet2 = jset2[0]
jnet2_indexed = jset2[1]
# Let's try to sort the jumps according to closest distance
# we don't want only the rotational jumps as listed.

def sortkey2(entry):
    jmp = jnet2[entry][0]
    or1 = mdbcontainer_fe.iorlist[jmp.state1.db.iorind][1]
    or2 = mdbcontainer_fe.iorlist[jmp.state2.db.iorind][1]
    dx = disp(mdbcontainer_fe, jmp.state1, jmp.state2)
    # c1 and c2 are always +1 for mixed dumbbell jumps.
    dx1 = np.linalg.norm(jmp.c1*or1/2.)
    dx2 = np.linalg.norm(dx + jmp.c2*or2/2. - jmp.c1*or1/2.)
    dx3 = np.linalg.norm(-jmp.c2*or2/2.)
    return dx1+dx2+dx3

z = np.zeros(3)
indices2 = []
indices_rot = []
for jt, jlist in enumerate(jnet2):
    if np.allclose(jnet2_indexed[jt][0][1], z):
        jmp = jlist[0]

        or1 = mdbcontainer_fe.iorlist[jmp.state1.db.iorind][1]
        or2 = mdbcontainer_fe.iorlist[jmp.state2.db.iorind][1]
        
        if np.allclose(np.dot(or1,or2)/(np.linalg.norm(or1)*np.linalg.norm(or2)), np.cos(np.pi/3.)):
#             print("got 60 deg jump at {}".format(jt))
            indices_rot.append(jt)
            continue
        elif np.allclose(np.dot(or1,or2), 0.):
#             print("got 90 deg jump at {}".format(jt))
            indices_rot.append(jt)
            continue
        else:
            continue
    indices2.append(jt)
ind_sort2 = sorted(indices2, key=sortkey2)[:2]
indices2all = ind_sort2[:1]
print(indices2all)

[12]


In [9]:
# check if we have the correct type of jumps
print(jnet2[ind_sort2[0]][0])

Jump object:
Initial state:
	Solute loctation:basis index = 0, lattice vector = [0 0 0]
	dumbbell : (i, or) index = 2, lattice vector = [0 0 0]
Final state:
	Solute loctation :basis index = 0, lattice vector = [-1 -1 -1]
	dumbbell : (i, or) index = 6, lattice vector = [-1 -1 -1]
Jumping from c1 = 1 to c2 = 1


In [10]:
for tup in mdbcontainer_fe.iorlist:
    print(tup)

(0, array([ 1.78190909e-01,  1.78190909e-01, -1.01947883e-17]))
(0, array([-1.78190909e-01, -1.78190909e-01,  1.01947883e-17]))
(0, array([ 0.17819091, -0.17819091,  0.        ]))
(0, array([ 0.        ,  0.17819091, -0.17819091]))
(0, array([-1.78190909e-01, -1.01947883e-17,  1.78190909e-01]))
(0, array([ 1.78190909e-01,  1.01947883e-17, -1.78190909e-01]))
(0, array([-1.01947883e-17,  1.78190909e-01,  1.78190909e-01]))
(0, array([ 1.01947883e-17, -1.78190909e-01, -1.78190909e-01]))
(0, array([0.17819091, 0.        , 0.17819091]))
(0, array([ 0.        , -0.17819091,  0.17819091]))
(0, array([-0.17819091,  0.17819091,  0.        ]))
(0, array([-0.17819091,  0.        , -0.17819091]))


In [11]:
jset2new = ([jnet2[i] for i in indices2all], [jnet2_indexed[i] for i in indices2all])

In [12]:
print(jset2new[0][0][0])

Jump object:
Initial state:
	Solute loctation:basis index = 0, lattice vector = [0 0 0]
	dumbbell : (i, or) index = 2, lattice vector = [0 0 0]
Final state:
	Solute loctation :basis index = 0, lattice vector = [-1 -1 -1]
	dumbbell : (i, or) index = 6, lattice vector = [-1 -1 -1]
Jumping from c1 = 1 to c2 = 1


In [13]:
# Initialize the calculator.
# We pass in the omega43 network we constructed above
start = time.time()
onsagercalculator = dumbbellMediated(pdbcontainer_fe, mdbcontainer_fe, jset0new, jset2new, 0.27,
                                     0.01, 0.01, 0.01, NGFmax=4, Nthermo=3)
print("onsager calculator initiation time = {}".format(time.time() - start))

initializing thermo
initializing kin
initializing NN
built shell 1: time - 0.042627573013305664
built shell 2: time - 0.8496289253234863
grouped states by symmetry: 1.0463166236877441
built mixed dumbbell stars: 0.0004315376281738281
built jtags2: 0.00037741661071777344
built mixed indexed star: 0.005850076675415039
building star2symlist : 8.225440979003906e-05
building bare, mixed index dicts : 0.0001556873321533203
2NN Shell initialization time: 2.8631811141967773

generating thermodynamic shell
built shell 1: time - 0.025104284286499023
built shell 2: time - 0.895559549331665
built shell 3: time - 4.299317121505737
grouped states by symmetry: 6.531683921813965
built mixed dumbbell stars: 0.0006096363067626953
built jtags2: 0.0004520416259765625
built mixed indexed star: 0.0055196285247802734
building star2symlist : 0.0001316070556640625
building bare, mixed index dicts : 0.0002009868621826172
thermodynamic shell generated: 17.91392993927002
Total number of states in Thermodynamic Sh

In [14]:
# next, we extract those omega43 jumps that we need.
jnet43 = onsagercalculator.jnet43
jnet43_indexed = onsagercalculator.jnet43_indexed
# Let's try to sort the jumps according to closest distance
# We assume that association and dissociation occurs via Johnson mechanism.

def sortkey3(entry):
    jmp = jnet43[entry][0] # This is an omega4 jump
    if not jmp.c2 == -1:
        print(c2)
    or1 = pdbcontainer_fe.iorlist[jmp.state1.db.iorind][1]
    or2 = mdbcontainer_fe.iorlist[jmp.state2.db.iorind][1]
    dx = disp4(pdbcontainer_fe, mdbcontainer_fe, jmp.state1, jmp.state2)
    # remember that c2 is -1 for an omega4 jump
    dx1 = np.linalg.norm(jmp.c1*or1/2.)
    dx2 = np.linalg.norm(dx - or2/2. - jmp.c1*or1/2.)
    dx3 = np.linalg.norm(jmp.c2*or2/2.)
    return dx1+dx2+dx3

z = np.zeros(3)
indices43 = []
for jt, jlist in enumerate(jnet43):
    if np.allclose(jnet43_indexed[jt][0][1], z):
        continue
    indices43.append(jt)    
ind_sort43 = sorted(indices43, key=sortkey3)[:1]
print(ind_sort43)

[19]


In [15]:
# check if we have the correct jump
print(jnet43[ind_sort43[0]][0])

Jump object:
Initial state:
	Solute loctation:basis index = 0, lattice vector = [0 0 0]
	dumbbell : (i, or) index = 0, lattice vector = [0 0 1]
Final state:
	Solute loctation :basis index = 0, lattice vector = [0 0 0]
	dumbbell : (i, or) index = 3, lattice vector = [0 0 0]
Jumping from c1 = 1 to c2 = -1


In [17]:
print(onsagercalculator.pdbcontainer.iorlist[0])
print(onsagercalculator.mdbcontainer.iorlist[3])

(0, array([ 1.78190909e-01,  1.78190909e-01, -1.01947883e-17]))
(0, array([ 0.        ,  0.17819091, -0.17819091]))


In [18]:
onsagercalculator.regenerate43(ind_sort43)

In [19]:
with open('FeCr_Onsg.pkl','wb') as fl:
    pickle.dump(onsagercalculator,fl)